## 준비

환경준비

In [16]:
# 호스트 머신에서 가상환경 구성
#uv venv && source .venv/bin/activate
#uv sync

mlx 형태로 모델 변환

In [ ]:
# mlx_lm 패키지를 사용하여 HF의 특정 모델을 지정된 경로/이름으로 mlx 모델로 변환
!uv run python -m mlx_lm.convert \
    --hf-path Qwen/Qwen2.5-0.5B-Instruct \
    --mlx-path models/qwen-0.5b

데이터 준비

In [18]:
# 원본데이터를 직접 다운받아서
!mkdir -p data
!curl -L -o data/ratings_train.txt https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!curl -L -o data/ratings_test.txt  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt

# 데이터셋을 로드한다
from datasets import load_dataset
data_files = {"train":"data/ratings_train.txt", "test":"data/ratings_test.txt"}
ds = load_dataset("csv", data_files=data_files, sep="\t")  # 컬럼: id, document, label
print(ds)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.9M  100 13.9M    0     0  27.0M      0 --:--:-- --:--:-- --:--:-- 27.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4778k  100 4778k    0     0  13.2M      0 --:--:-- --:--:-- --:--:-- 13.2M


Generating train split: 150000 examples [00:00, 540720.03 examples/s]
Generating test split: 50000 examples [00:00, 511634.01 examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [19]:
# 파케 형태로 변환해둔다
ds["train"].to_parquet("data/nsmc-train.parquet")
ds["test"].to_parquet("data/nsmc-test.parquet")

Creating parquet from Arrow format: 100%|██████████| 50/50 [00:00<00:00, 1949.10ba/s]


5341357

데이터는 다음과 같은 형태로 준비되어 있다
- 데이터 ID
- 영화감상평
- 긍정/부정 라벨 (1이면 긍정, 0이면 부정)

In [20]:
import json, random, os

In [21]:
def make_chat(example):
    txt = example["document"]
    lab = "긍정" if example["label"]==1 else "부정"
    # 간단한 지시문 프롬프트(분류 태스크를 대화형으로)
    messages = [
        {"role":"system","content":"너는 한국어 감성분석 어시스턴트야."},
        {"role":"user","content":f"다음 리뷰의 감성을 한 단어로만 답해줘(긍정/부정). 리뷰: {txt}"},
        {"role":"assistant","content":lab}
    ]
    return {"messages": messages}

def dump(ds, path, k):
    data = [make_chat(x) for x in ds.select(range(min(k, len(ds))))]
    with open(path,"w",encoding="utf-8") as f:
        for r in data: f.write(json.dumps(r, ensure_ascii=False)+"\n")

dump(ds["train"], "data/nsmc_train_10k.jsonl", 10_000)
dump(ds["test"], "data/nsmc_test_2k.jsonl", 2_000)
print("ok")


ok


## LoRA 튜닝

- **LoRA** (Low Rank Adaptation)
  - 대형 언어모델의 원본 가중치는 고정하고, 각 선형층에 저랭크 보정만 학습하는 미세조정 기법
  - 전체 파인튜닝 대비 학습 파라미터 수, VRAM, 시간을 절약할 수 있고
  - 어댑터만 저장/배포하면 되어 경량 전송/버전관리에 이점이 있음
  - 추론시 W` = W + AB로 계산하여 원본은 그대로, LoRA만 추가하는 방식
  - 작업, 도메인에 맞춘 소규모 데이터로 빠르게 적용 가능
  - 비용을 절감하고 속도를 높일 수 있으며, 여러 작업용 어댑터를 모듈처럼 교체 가능
  - 베이스 모델이 이미 잘 하는 작업이라면 개선폭이 낮고, 데이터가 편향/협소하면 과적합 위험
  - 항상 baseline과 지표 비교로 효과를 검증해야힘

lora에 맞춰 train, valid, test로 데이터 준비

In [38]:
!mkdir -p data/lora
!head -n 9500  data/nsmc_train_10k.jsonl > data/lora/train.jsonl
!tail -n 500   data/nsmc_train_10k.jsonl > data/lora/valid.jsonl
!cp data/nsmc_test_2k.jsonl              data/lora/test.jsonl

mlx_lm lora의 주요 옵션 사용방법 정리

- 핵심 옵션
  - `--model <PATH|HF_ID>`
    미세조정의 **베이스 모델**을 지정.
    - 예: `models/qwen-0.5b`(로컬 변환본) 또는 `Qwen/Qwen2.5-0.5B-Instruct`(HF ID).
    - 로컬 변환본이면 네트워크 없이 빠르고 안정적.
  - `--adapter-path <DIR>`
    **LoRA 어댑터 가중치 출력 위치**.
    - 학습 중/종료 시 여기에 체크포인트가 저장됨.
    - 여러 실험을 병행할 땐 경로를 실험명으로 구분(예: `adapters/nsmc-qwen0p5b-r8`).
  - `--train`
    **학습 모드**를 켜는 스위치. 없으면 학습이 시작되지 않음.
  - `--data <DIR|FILE>`
    **학습/검증/테스트 데이터 위치**.
    - 디렉터리를 주면 내부의 `train.jsonl`, `valid.jsonl`, (선택) `test.jsonl`을 자동 인식.
  - `--batch-size <INT>`
    **최대 토큰 길이 × 배치 크기**가 메모리에 크게 영향.
    - OOM 나면 `4 → 2`로 줄이거나, `--max-seq-length`로 시퀀스 길이를 낮추는 방식도 가능.
  - `--iters <INT>`
    **총 학습 스텝 수**. `epochs` 대신 쓰는 개념.
    - 근사 환산: `steps ≈ (train_samples / batch_size) × epochs`
    - 예) 10k 샘플, batch 8, 2 epochs → 약 2,500 steps.
  - `--learning-rate <FLOAT>`
    학습률. LoRA는 보통 `1e-4 ~ 3e-4` 사이에서 시작해 탐색.
  - `--steps-per-report <INT>`
    **로그/손실 보고 주기**(몇 스텝마다 콘솔에 학습 상태 출력할지). 너무 작으면 콘솔 스팸, 너무 크면 진행 파악이 어렵다 보통 20~50 권장.
  - `--steps-per-eval <INT>`
    **검증(Validation) 실행 주기**. 지정 스텝마다 `valid.jsonl` 일부로 평가.
    - 빠른 루프 확인용으로 100~500 사이가 실용적.
  - `--val-batches <INT>`
    검증 시 **몇 배치**를 사용할지.
    - 전체 검증을 매번 다 돌리면 느리니, 20~100 같은 범위로 “샘플링 평가”를 자주 하고, 마지막에 전체 평가를 별도로 돌리는 전략이 좋음.
  - `--seed <INT>`
    시드 고정. **재현성**을 위해 반드시 넣어두는 게 좋음(데이터 셔플·샘플링에 영향).

- 자주 쓰는 보조 옵션
  - `--fine-tune-type {lora,dora,full}`
    **튜닝 방식** 선택. 기본은 LoRA. DORA/Full fine-tune은 자원 요구량↑.
  - `--optimizer {adam,adamw,sgd,adafactor}`
    옵티마이저 선택. LoRA에선 `adamw`가 무난.
    - **SGD (Stochastic Gradient Descent)**: 가장 기본적인 최적화. 단순히 기울기만 보고 이동. 느리지만 단순/안정.
    - **Adam**: 각 파라미터마다 학습률을 적응적으로 조절(모멘텀 + RMSProp). 대부분 기본값.
    - **AdamW**: Adam + Weight Decay(가중치 감소 정규화). 과적합 방지에 더 효과적.
    - **Adafactor**: 메모리 효율 극대화(대형 LM 튜닝에서 사용). Adam보다 가볍지만 불안정할 때도 있음.
  - `--mask-prompt`
    프롬프트 토큰 쪽 손실을 **마스킹**해 정답(assistant 부분)에 집중해 학습. 대화형 데이터에 유리.
    - 대화 데이터는 `system` + `user` + `assistant`가 섞여 있음.
    - 우리가 원하는 건 모델이 **“assistant 부분”만 잘 예측**하는 것.
    - `mask-prompt`는 system/user 토큰의 loss를 0으로 만들어, 오로지 **assistant 응답 부분에 대해서만 학습**하도록 합니다.
    - 안 쓰면 모델이 불필요하게 “user 메시지까지 흉내”내려다 성능이 떨어질 수 있어요.
  - `--num-layers <INT>`
    LoRA를 **상위 N개 레이어**에만 적용(자원 절약용). 모델 종속이라 과도하면 성능 하락 가능.
    - LoRA는 모든 레이어에 보정행렬을 붙일 수 있지만,
    - “상위 몇 개 레이어”만 적용해도 성능이 크게 올라가는 경우가 많습니다.
    - 따라서 **num-layers=N**으로 제한하면 메모리/속도를 아끼면서 효과적인 튜닝이 가능합니다. (예: 24레이어 중 마지막 8개에만 LoRA 적용)
  - `--max-seq-length <INT>`
    최대 시퀀스 길이. 길수록 메모리↑. OOM 시 2048 → 1024로 낮추는 식으로 조정.
  - `--save-every <INT>` / `--resume-adapter-file <PATH>`
    **주기적 저장/재개**. 긴 러닝에서 유용. 중단 후 이어달리기에 사용.
  - `--test` / `--test-batches <INT>`
    학습 대신 **테스트 모드**로 `test.jsonl`을 빠르게 스캔해 품질을 훑어봄(정밀 지표는 별도 스크립트 추천).
  - `--grad-checkpoint`
    **그라드 체크포인팅**으로 메모리 사용을 줄이는 대신 연산량↑(속도 약간 느려짐).
    - **grad-checkpoint**
      - 역전파(backprop)에서 중간 계산 결과를 다 저장하지 않고, 필요한 시점에 **다시 계산**해서 메모리 절약하는 기법.
      - 메모리↓, 속도는 조금 느려짐.
      - GPU/Apple Silicon 메모리가 빡빡할 때 켜면 유용.
    - **save-every**
      - 학습 중 **몇 step마다 LoRA 어댑터를 디스크에 저장할지**.
      - 장시간 학습하다 중단되더라도 여기서부터 재개 가능.
      - 학습 알고리즘과 관계없고, 단순히 **체크포인트 파일 주기**에 해당.
  - `--wandb <PROJECT_NAME>`
    Weights & Biases 로깅 연동.

- 운용 팁
  - **데이터 디렉터리 구조 필수**: `--data data/lora/` 안에 `train.jsonl`, `valid.jsonl`(필수), `test.jsonl`(선택).
  - **iters ↔ 시간 감**: 10k/bs8이면 1 epoch~1,250 steps 정도. M1 기준 1,500~2,500 steps가 “1회 완주 체감”에 적당.
  - **검증 빈도**: `--steps-per-eval 200`, `--val-batches 50` 정도면 10k 소셋에서 과하지 않음.
  - **로그 가독성**: `--steps-per-report 20~50` 권장.
  - **베이스라인 비교**: 최종 평가는 `eval.py`(baseline vs adapter)로 **ACC/F1**을 같은 조건에서 두 번 찍어 **개선폭**을 기록.

> - **경사하강법이란?**
>   - 머신러닝 모델은 보통 “손실 함수(loss)” 라는 성능 측정 식이 있습니다.
>   - 예: 예측 값과 정답 사이의 차이(오차).
>   - 우리가 원하는 건 손실이 최소가 되는 지점(최적 파라미터).
>   - 그런데 수학적으로 한 번에 답을 구할 수 없을 때가 대부분 → 점진적으로 오차를 줄여 나가는 방법이 필요합니다.
>   - 바로 그게 경사하강법이에요.
>
> - “경사(gradient)” = 현재 위치에서 손실이 얼마나, 어느 방향으로 증가하는지를 보여주는 기울기(미분 값).
> - “하강(descent)” = 손실이 커지는 방향 반대로 조금씩 이동.
> - 즉, 가중치를 기울기 반대 방향으로 조금씩 바꿔나가면서 손실을 줄여나가는 과정입니다.

In [40]:
!mkdir -p adapters
!uv run python -m mlx_lm lora \
  --model models/qwen-0.5b \
  --adapter-path adapters/nsmc-qwen0p5b-r8 \
  --train \
  --data data/lora \
  --batch-size 8 \
  --iters 2500 \
  --learning-rate 2e-4 \
  --steps-per-report 20 \
  --steps-per-eval 200 \
  --val-batches 50 \
  --seed 42

Loading pretrained model
Loading datasets
Training
Trainable parameters: 0.073% (0.360M/494.033M)
Starting training..., iters: 2500
Calculating loss...: 100%|██████████████████████| 50/50 [00:38<00:00,  1.29it/s]
Iter 1: Val loss 4.573, Val took 38.844s
Iter 20: Train loss 2.032, Learning Rate 2.000e-04, It/sec 0.629, Tokens/sec 427.885, Trained Tokens 13606, Peak mem 4.479 GB
Iter 40: Train loss 1.395, Learning Rate 2.000e-04, It/sec 0.723, Tokens/sec 468.038, Trained Tokens 26560, Peak mem 4.480 GB
Iter 60: Train loss 1.403, Learning Rate 2.000e-04, It/sec 0.675, Tokens/sec 453.783, Trained Tokens 40013, Peak mem 4.480 GB
Iter 80: Train loss 1.320, Learning Rate 2.000e-04, It/sec 0.751, Tokens/sec 488.534, Trained Tokens 53028, Peak mem 4.480 GB
Iter 100: Train loss 1.396, Learning Rate 2.000e-04, It/sec 0.688, Tokens/sec 462.891, Trained Tokens 66489, Peak mem 4.480 GB
Iter 100: Saved adapter weights to adapters/nsmc-qwen0p5b-r8/adapters.safetensors and adapters/nsmc-qwen0p5b-r8/000

## 결과확인

테스트

- `--adapter-path`: 모델에 합성할 LoRA 가중치
- `--test`: 손실만 계산하는 루프 실행
- `--test-batches`: 루프 최대값

In [ ]:
# Test the LoRA adapter
!uv run python -m mlx_lm lora \
  --model models/qwen-0.5b \
  --adapter-path adapters/nsmc-qwen0p5b-r8 \
  --test \
  --data data/lora \
  --test-batches 250

Loading pretrained model
Loading datasets
Testing
Calculating loss...: 100%|████████████████████| 250/250 [01:23<00:00,  2.98it/s]
Test loss 1.278, Test ppl 3.590.


In [45]:
# Test the baseline
!uv run python -m mlx_lm lora \
    --model models/qwen-0.5b \
    --adapter-path "" \
    --test \
    --data data/lora \
    --test-batches 250


Loading pretrained model
Loading datasets
Testing
Calculating loss...: 100%|████████████████████| 250/250 [01:21<00:00,  3.05it/s]
Test loss 4.585, Test ppl 98.004.


- **테스트결과**
  - LoRA 어댑터 적용: Test Loss 1.278 / Perplexity 3.50
  - BaseLine : Test Loss 4.585 / Perplexity 98.0
  - ppl이 낮아짐, 튜닝후 정답 토큰을 3.59개 후보 중 하나 정도로 봄
  - 태스크 정확도(ACC/F1)는 따로 평가

평가

In [53]:
from mlx_lm import load
model, tok = load("models/qwen-0.5b")

print(tok.encode("긍정"))
print(tok.encode("부정"))

[144203, 29281]
[63089, 29281]


In [58]:
import json

text = ""
messages = [
    {"role": "system", "content": "너는 한국어 감성분석 어시스턴트야."},
    {"role": "user", "content": f"다음 리뷰의 감성을 한 단어로만 답해줘(긍정/부정). 리뷰: {text}"}
]
prompt = ", ".join([json.dumps(m, ensure_ascii=False) for m in messages])
print(prompt)

{"role": "system", "content": "너는 한국어 감성분석 어시스턴트야."}, {"role": "user", "content": "다음 리뷰의 감성을 한 단어로만 답해줘(긍정/부정). 리뷰: "}


In [ ]:
# scripts/eval.py (수정본)
import re, json, tqdm
from datasets import load_dataset
from mlx_lm import load, generate

MODEL_PATH = "models/qwen-0.5b"
ADAPTER_PATH = "adapters/nsmc-qwen0p5b-r8"
TEST_PATH = "data/lora/test.jsonl"
IDX=0

def extract_text_and_label(rec):
    # chat 형식: system/user/assistant 중 user와 assistant만 사용
    msgs = rec["messages"]
    user_text = next((m["content"] for m in msgs if m["role"] == "user"), "")
    gold = next((m["content"] for m in msgs if m["role"] == "assistant"), "").strip()
    # 혹시 공백/장식이 섞여도 '긍정|부정'만 남기기
    gold = "긍정" if "긍정" in gold else "부정"
    return user_text, gold

def ask(text, model, tokenizer):
    global IDX
    IDX+=1
    prompt = (
        "너는 한국어 감성분석 어시스턴트야.\n"
        "다음 리뷰의 감성을 딱 한 단어로만 출력해. 다른 말/기호 금지.\n"
        "가능한 답: 긍정 또는 부정\n"
        f"리뷰: {text}\n"
        "정답:"
    )
    out = generate(model, tokenizer, prompt=prompt, max_tokens=4)
    m = re.search("(긍정|부정)", out)
    if (IDX%2000)<5:
        print(f"real output: {out}, normalized: {m}")
    return m.group(1) if m else None   # ★ fallback을 None으로

def evaluate(model, tokenizer, ds, limit=2000):
    ds = ds.select(range(min(limit, len(ds))))
    ok = 0
    ok, total, unclassified = 0, 0, 0
    for rec in tqdm.tqdm(ds):
        text, gold = extract_text_and_label(rec)
        pred = ask(text, model, tokenizer)
        if pred is None:
            unclassified += 1
        else:
            total += 1
            ok += (pred == gold)
    
    acc = ok/total if total > 0 else 0
    print(f"ACC={acc:.4f} (on {total} classified samples)")
    print(f"Unclassified={unclassified} / {len(ds)}")
    return ok / len(ds)

def main():
    ds_te = load_dataset("json", data_files={"test": TEST_PATH})["test"]
    base_model, base_tok = load(MODEL_PATH, adapter_path=ADAPTER_PATH)
    acc_base = evaluate(base_model, base_tok, ds_te, limit=2000)
    print(f"Baseline ACC={acc_base:.4f}")

if __name__ == "__main__":
    main()

/Users/bachtaeyeong/10_SrcHub/AI-Service-Lab/week01-model-finetune/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BaseLine ---------------------------------------------


  0%|          | 2/2000 [00:00<06:11,  5.39it/s]

real output: 부정

이, normalized: <re.Match object; span=(0, 2), match='부정'>
real output: 부정

이, normalized: <re.Match object; span=(0, 2), match='부정'>


  0%|          | 4/2000 [00:00<05:54,  5.64it/s]

...

real output: 부정. �, normalized: <re.Match object; span=(0, 2), match='부정'>
real output: 부정. , normalized: <re.Match object; span=(0, 2), match='부정'>


  0%|          | 4/2000 [00:00<05:59,  5.56it/s]

real output: 부정 1, normalized: <re.Match object; span=(0, 2), match='부정'>
real output: 부정... , normalized: <re.Match object; span=(0, 2), match='부정'>


100%|██████████| 2000/2000 [06:17<00:00,  5.30it/s]

real output: 부정. , normalized: <re.Match object; span=(0, 2), match='부정'>
ACC=0.6379 (on 1936 classified samples)
Unclassified=64 / 2000
LoRA ACC=0.6175


- **1차 평가결과**
- NSMC 감성 분류 (Qwen-0.5B, M1 Pro, LoRA 튜닝)

    | 모델          | Test Loss | Test PPL | ACC (2000 샘플) | 소요 시간 |
    |---------------|-----------|----------|-----------------|-----------|
    | Baseline      | 4.585     | 98.00    | 0.2775          | 5m 49s    |
    | LoRA (r=8)    | 1.278     | 3.59     | 0.0010          | 6m 02s    |

> - `LoRA의 r=8` : 
>   - 작으면 적은 메모리로 빠른학습, 크면 더 많은 표현력이며 느리고 메모리 더 필요. 
>   - r=8이면 dxd 행렬 대신 2xdx8짜리 행렬 학습.
> - `Test Loss`:
>   - 모델이 정답 분포와 얼마나 멀리 떨어져 있는지, 여기서는 cross-entropy loss로 계산
>   - 값이 작을 수록 모델이 "정답 확률에 더 높은 확률을 준다는 뜻"
> - `Perplexity(PPL)`: 
>  - 모델이 다음 토큰을 맞추는 데 평균적으로 몇 가지 선택지 사이에서 헷갈리는지
>  - ppl=1이면 항상 정답만, ppl=3.5면 3-4개 후보중, ppl=98이면 거의 랜덤

- **2차 평가결과**
  - 1차 평가중 생성된 데이터를 출력해보니 불필요한 텍스트를 자꾸 덧붙임.
  - 또한 긍정적/부정적이다 등 분류자체는 성공했으나 형식상 실패도 보임
  - 프롬프트를 수정하여 포맷을 일치화하도록 하고, 정규식 사용해 형식면에서도 수정
    | 모델          | Test Loss | Test PPL | ACC (2000 샘플) | 소요 시간 |
    |---------------|-----------|----------|-----------------|-----------|
    | Baseline      | 4.585     | 98.00    | 0.4910          | 5m 58s    |
    | LoRA (r=8)    | 1.278     | 3.59     | 0.6175          | 5m 30s    |

중간 체크포인트중 후보 선택

In [3]:
!uv run python -m mlx_lm lora \
  --model models/qwen-0.5b \
  --resume-adapter-file adapters/nsmc-qwen0p5b-r8/0001600_adapters.safetensors \
  --adapter-path adapters/nsmc-qwen0p5b-r8 \
  --test \
  --data data/lora \
  --test-batches 250
  
!uv run python -m mlx_lm lora \
  --model models/qwen-0.5b \
  --adapter-path adapters/nsmc-qwen0p5b-r8 \
  --resume-adapter-file adapters/nsmc-qwen0p5b-r8/0002000_adapters.safetensors \
  --test \
  --data data/lora \
  --test-batches 250
  
!uv run python -m mlx_lm lora \
  --model models/qwen-0.5b \
  --adapter-path adapters/nsmc-qwen0p5b-r8 \
  --resume-adapter-file adapters/nsmc-qwen0p5b-r8/0002400_adapters.safetensors \
  --test \
  --data data/lora \
  --test-batches 250

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Loading datasets
Testing
Calculating loss...: 100%|████████████████████| 250/250 [01:23<00:00,  2.98it/s]
Test loss 1.278, Test ppl 3.590.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Loading datasets
Testing
Calculating loss...: 100%|████████████████████| 250/250 [01:23<00:00,  3.01it/s]
Test loss 1.278, Test ppl 3.590.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Loading datasets
Testing
Calculating loss...: 100%|████████████████████| 250/250 [01:23<00:00,  2.99it/s]
Test loss 1.278, Test ppl 3.590.
